In [11]:
import openai

In [43]:
import pandas as pd

df = pd.read_csv("synthetic_data.csv")

df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.drop_duplicates(inplace=True)
df = df.head()
df.head()

,Name,Address,Email,Phone Number,Date,Salary,Company,Messy Text
0,Emma Davis,"327 Broadway, New York, NY",emmadavis83@outlook.com,123.456.7890,01-02-2024,500000.0,Facebook,Can yu pleas clarrify ur qustion?
1,John Doe,"927 1st Ave, Los Angeles, CA",johndoe54@invalid,123-456-7890,"Feb 1, 2024",55000.0,Amazon Corp.,Can yu pleas clarrify ur qustion?
2,John Doe,"354 Market St, Austin, TX",johndoe44@gmail.com,123-456-7890,1st Feb 2024,500000.0,Microsoft,Can yu pleas clarrify ur qustion?
3,Emily Johnson,"112 Elm St, Los Angeles, CA",emilyjohnson14@outlook.com,(123) 456-7890,2024/02/01,150000.0,Apple,I lovee to wrk at google!
4,James White,"337 Broadway, Austin, TX",jameswhite82@outlook.com,(123) 456-7890,"Feb 1, 2024",500000.0,Amazon,tesla is a great cmpany but I wud prefer Meta.


In [10]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
def resolve_entity(text):
    prompt = f"Resolve entity variations to a standardized form: {text}"
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

df['standardized_company'] = df['company_name'].apply(resolve_entity)


In [50]:
from dotenv import load_dotenv

load_dotenv()

import os
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [38]:
column_name = "company"
column_information = "name of the company"

text = "nvidia corp"
# text = "Bajaj allimc"

In [ ]:
prompt = f"""I am giving you text field from the column titled {column_name}. 
            It contains inform about {column_information} 
            Resolve it to to a standardized form but do not modify if you are not sure of the format
            for the specific data: {text}. 
            Give no preambles, just the result."""
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt
        }
    ]
)

In [40]:
response.choices[0].message.content

'Nvidia Corporation'

In [32]:
response.choices[0].message.content

'Swapnil Financials'

In [46]:
def standardize_column(text, column_name, column_information):
    prompt = f"""I am giving you text field from the column titled {column_name}. 
                It contains inform about {column_information} 
                Resolve it to to a standardized form but do not modify if you are not sure of the format
                for the specific data: {text}. 
                Give no preambles, just the result."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ]   
    )
    return response.choices[0].message.content

In [47]:
column_name = "company"
column_information = "name of the company"
df['standardized_company'] = df['Company'].apply(standardize_column, args=(column_name, column_information))

In [48]:
df['standardized_company']

0           Facebook, Inc.
1       Amazon Corporation
2    Microsoft Corporation
3               Apple Inc.
4                   Amazon
Name: standardized_company, dtype: object

In [49]:
df['Company']

0        Facebook
1    Amazon Corp.
2       Microsoft
3           Apple
4          Amazon
Name: Company, dtype: object